In [1]:
import ecdsa
import random
import json
import numpy as np
import time
import hashlib
from ecdsa import SigningKey,VerifyingKey,BadSignatureError
import asyncio
import heapq

In [2]:
MIN_AMOUNT=1000
MAX_AMOUTN=10000
MIN_HASHRATE=1000
MAX_HASHRATE=10000
MASTETNODES_NUMS=100
REGION_NUMS=32
POW_NUMS=21
VERIFY_NUMS=11
UTXO_WEIGHT_DELTA=0.1
MAX_NONCE = 2 ** 32
VERIFY_TIME=10
MAX_BLOCKHEIGHT=100
ELECT_GAP=10
VERIFY_GAP=25
POW_DIFF=15
MINUPTIME=3600

In [3]:
class BlockChain:
    def __init__(self):
        self.blockheight=0
        self.version='00000010'
        self.prevblockhash='00000000000000000000000000000701'
        self.merkleroot='00000000000000000000000000001994'
        self.timestamp=hex(int(time.time()))[2:]
        self.target='00000010'
        self.blockheader=self.getBlockheader()
    
    def getBlockheader(self):
        return self.version+self.prevblockhash+self.merkleroot+self.timestamp+self.target
        
    def getDifficulty(self):
        diff=POW_DIFF
        return diff
    
    def getPrevblockhash(self):
        return self.prevblockhash
    
    def getBlockheight(self):
        return self.blockheight
    
    def nextBlock(self,blockhash):
        self.blockheight+=1
        self.version=self.version
        self.prevblockhash=blockhash
        self.merkleroot=''.join(random.choice('0123456789abcdef') for _ in range(32))
        self.timestamp=hex(int(time.time()))[2:]
        self.target=self.target
        self.blockheader=self.getBlockheader()

In [4]:
class Masternode(BlockChain):
    def __init__(self,utxoamount,hashrate,externalip,privkey,txid):
        super(Masternode,self).__init__()
        self.hashrate=hashrate                #节点宣称算力
        self.externalip=externalip            #节点IP
        self.privkey=privkey                  #节点私钥 - 不广播
        self.txid=txid                        #节点txid - 验证抵押金，识别ID
        self.utxoamount=utxoamount            #节点抵押货币
        self.publickey=self.generatePublickey(self.privkey) #节点公钥

    def generatePublickey(self,privkey):
        return privkey.get_verifying_key()
        
    def proof_of_work(self,header, difficulty_bits):
        target = 2 ** (256 - difficulty_bits)
        for nonce in range(MAX_NONCE):
            hash_result = hashlib.sha256((str(header) + str(nonce)).encode('utf-8')).hexdigest()
            if int(hash_result, 16) < target:
                return (hash_result, nonce)
        return False
                
    async def mining(self):
        blockheader=self.getBlockheader()
        difficulty_bits=self.getDifficulty()
        hash_result,nonce=self.proof_of_work(blockheader,difficulty_bits)
        await asyncio.sleep(random.uniform(0,1))
        return hash_result,nonce,time.time()

    async def proveHashrate(self):
        blockheader=self.getPrevblockhash+self.txid
        endblock=self.getBlockheight()+VERIFY_TIME
        nonces=[]
        while self.getBlockheight()<endblock:
            blockhash,nonce=self.proof_of_work(blockheader,difficulty_bits)
            blockheader=blockhash
            nonces.append(nonce)
        await asyncio.sleep(random.uniform(0,1))
        return nonces

    def verifyHashrate(self,node,nonces):
        header=self.blockhash+node.txid
        for nonce in nonces:
            hash_result = hashlib.sha256((str(header) + str(nonce)).encode('utf-8')).hexdigest()
            if int(hash_result, 16) > target:
                return False
            header=hash_result
        return True
    
    def sign(self,message):
        return self.privkey.sign(message)
    
    def verify(self,signature,message,verifykey):
        return verifykey.verify(signature,message)
    
    def verifyStake(self,node):
        is_node_txid_upsent=True
        is_txid_more_than_stake=True
        return is_node_txid_upsent*is_txid_more_than_stake
    
    def ping(self,node,mn):                                            #节点间连接速度
        return random.uniform(MIN_PING,MAX_PING)
    
    def chosePartners(self,node,PartMN):                               #邀请小组成员
        connectivity={}
        if len(PartMN)<=2:
            return list(PartMN)
        available_mn = [mn for mn in PartMN if abs(int(mn.txid,16)-int(node.txid,16)+int(self.blockhash,16))<2**10 and mn!=node]
        if len(available_mn)<=2:available_mn=[mn for mn in PartMN if mn!=node]
        for mn in available_mn:
            connectivity[mn]=self.ping(node,mn)
        sorted_connectivity=sorted(connectivity.items(), key=lambda item:item[1])
        return [node,sorted_connectivity[0][0],sorted_connectivity[1][0]]


In [5]:
class ListNode(Masternode):
    def __init__(self,utxoamount,hashrate,externalip,privkey,txid,uptime):
        super(ListNode,self).__init__(utxoamount,hashrate,externalip,privkey,txid)
        self.uptime=uptime                    #节点上线时间
        self.level=0                          #节点选举等级
        self.region= -1
        self.father=None                      #父节点
        self.children=None                    #子节点
        
    def updateLevel(self,level):
        self.level=level
    
    def updateRegion(self,region):
        self.region=region
        
    def updateFather(self,father):
        self.father=father

    def updateChildren(self,children):
        self.children=children
            
    def updateUptime(self,uptime):
        self.uptime=uptime

In [6]:
class Matrix(BlockChain):
    def __init__(self):
        super(Matrix,self).__init__()
        self.global_list=self.generateMN(MASTETNODES_NUMS)
        self.pow_nodes=random.sample(list(self.global_list.values()),21)
        self.verify_nodes=random.sample(list(self.global_list.values()),11)
        
    def generateMN(self,nums):          #生成主节点列表
        global_list = {}
        for i in range(nums):
            utxoamount = random.randint(MIN_AMOUNT, MAX_AMOUTN)
            hashrate = random.uniform(MIN_HASHRATE, MAX_HASHRATE)
            externalip = '.'.join([str(random.randint(0, 255)) for _ in range(4)])
            privkey = SigningKey.generate()
            txid= ''.join(random.choice('0123456789abcdef') for _ in range(32)) 
            uptime = time.time()-random.randint(0,100000)
            global_list[txid]=ListNode(utxoamount,hashrate, externalip, privkey, txid,uptime)
        return global_list
    
    def clearAll(self):
        for node in self.global_list.values():
            node.updateLevel(0)
            node.updateChildren(None)
            node.updateFather(None)
            node.updateRegion(-1)

    def dividedRegions(self):
        regions={}
        region_num=0
        for item in sorted(self.global_list.items(),key=lambda item:item[1].utxoamount*item[1].hashrate):
            item[1].updateRegion(region_num)
            region_num=(region_num+1)%REGION_NUMS
            if region_num not in regions:
                regions[region_num]=[] 
            regions[region_num].append(item[1])
        return regions
    
    def groupPK(self,group):
        timestamp=int(self.timestamp,16)
        utxo_weights=[max(float(node.utxoamount)*(timestamp-node.uptime)*node.hashrate,UTXO_WEIGHT_DELTA) for node in group]
        p=[float(w)/sum(utxo_weights) for w in utxo_weights]
        winner=np.random.choice(group,1,p)[0]
        return winner
        
    def regionElections(self,region):
        select_pool=[]
        random.seed(int(self.prevblockhash[:8],16))
        rounds=0
        while True:
            for node in region:
                if node.level==rounds: #and self.timestamp-node.uptime>MINUPTIME:
                    select_pool.append(node)
            if len(select_pool)==1:
                winner=select_pool[0]
                winner.updateUptime(winner.uptime+(int(self.timestamp,16)-winner.uptime)/2)
                return select_pool[0]
            while select_pool:
                if len(select_pool)>=3:
                    group=random.sample(select_pool,3)
                else:
                    group=select_pool
                winner=self.groupPK(group)
                children=[node for node in group if node!=winner]
                winner.updateLevel(rounds+1)
                winner.updateChildren(children)
                for node in children:
                    node.updateFather(winner)
                for node in group:
                    select_pool.remove(node)
            rounds+=1
        return False
                
            
    def elect(self):
        self.clearAll()
        regions=self.dividedRegions()
        pow_nodes=[]
        verify_nodes=[]
        for i in range(REGION_NUMS):
            if i<POW_NUMS:
                pow_nodes.append(self.regionElections(regions[i]))
            else:
                verify_nodes.append(self.regionElections(regions[i]))
        return pow_nodes,verify_nodes
    
    async def powNextblock(self):
        tasks=[node.mining() for node in self.pow_nodes]
        results=await asyncio.gather(*tasks)
        winner=-1
        blockhash=-1
        mintime=time.time()
        for i in range(POW_NUMS):
            hash_result,nonce,timestamp=results[i]
            if timestamp<mintime:
                blockhash,winner,mintime=hash_result,i,timestamp
        return blockhash,winner
    
    async def verifyAll(self):
        node.p
    
    def run(self):
        for blockheight in range(MAX_BLOCKHEIGHT):
            loop=asyncio.new_event_loop()
            blockhash,winner=loop.run_until_complete(self.powNextblock())
            loop.close()
            if blockheight%ELECT_GAP==7:
                self.pow_nodes,self.verify_nodes=self.elect()
                print(''.join('-' for _ in range(50)))
                print('POW_NODES: '+' '.join(node.txid for node in self.pow_nodes))
                print('VERYFI_NODES: '+' '.join(node.txid for node in self.verify_nodes))
                print(''.join('-' for _ in range(50)))
            if blockheight%VERIFY_GAP==3:
                pass
            self.nextBlock(blockhash)
            for node in self.global_list.values():
                node.nextBlock(blockhash)
            print('blockheight: '+str(self.blockheight),'winner: '+self.pow_nodes[winner].txid,'blockhash: '+blockhash)

In [7]:
matrix=Matrix()
matrix.run()

blockheight: 1 winner: b8554490c37801b91ee5509b32364b90 blockhash: 0001a05209f38b86422c785d4f9ca51539686cee6174f7a77af99505a8317f9c
blockheight: 2 winner: f0f36615579c8e23ca6c11671ae7213b blockhash: 0000361ee8ef4805bfd6fb1f55e13c547312059d499cfe75e66f54874732707d
blockheight: 3 winner: 8ce7576cc7ffffd85e0c619c54121542 blockhash: 000051fcf4fc7ab8613adcc6e64ae5bbce71009a6b8fde8812bc7a7fdcc2b077
blockheight: 4 winner: f0f36615579c8e23ca6c11671ae7213b blockhash: 0001d249acde87e0684f14717c344c773c737ac799101dab9567943dded153d9
blockheight: 5 winner: 7478b3926b58c836436f145d6140d967 blockhash: 0000406b1922f0a879447c6082b441c6f0f1105f9d439f75a9496f537cc29ea0
blockheight: 6 winner: 1a2d7e4bdc48de3854a16387f12db2ec blockhash: 0000f268e8fc747a759513f30574fc11d40f71dd788869165ff548aa25bd5e97
blockheight: 7 winner: 8849dbd4fa9e19afa6705a05337eb7dd blockhash: 0001e763bd2477614a9e1edf7ec724a56f60f755621d6ef8ca82cb396d14910f
--------------------------------------------------
POW_NODES: af15d2a6cc7c52

blockheight: 36 winner: 216545b14e2fd0c3754a52ad24966f3d blockhash: 0001d72fff16a2251b953e565af088bce3d0f90b0fc45111e642f9eee418eb17
blockheight: 37 winner: 417bb4ed50894b21207add3526caa834 blockhash: 00007e3f450685c2091aa971a2a1a528b856bc5ebec9c021f5c865ee1eb0879f
--------------------------------------------------
POW_NODES: 5294c1ebe2080717c2c9d737490a01f8 7748a130899903a079a4d630674ae892 d1b76e1c84e64051506efdd21810fa32 d80acc658a535f45c4a3b16136be6f74 097e9804b11ec53f67beff70279b7e2c 89bd8727266a9671aa276ca0f121a77f e26fcb9daf7c1de62137b5454b4cf555 08fea26bf6e6c5df11838b807a338999 2a6669e7f7993c819533877c1455dd7d 417bb4ed50894b21207add3526caa834 c8e76a9a71ea86150aeed68c32a5507c 40f8329121414e071840d5ce0e826262 b9a4eae1fddeddf713c775d7ff43ff99 b8554490c37801b91ee5509b32364b90 22f7cfb4c5902d998ec623fe715b81a3 99cdd208b8bafae050bb66e786e5d5d1 86f3b81c06303cd8a933704846eef197 01073ee15582cd5b489ce252671244a7 1268e833c4643187545355f9bdee1795 7ce4d1570fd82f4ae2805ba0c4c0cb16 cffe488e8713

blockheight: 69 winner: 323c63c511ff3330fe60026eb999a03c blockhash: 0001ebb0ea384af19a11306f7771f1c7c4c720d9bef986e0ca25b37b373916b8
blockheight: 70 winner: b9a4eae1fddeddf713c775d7ff43ff99 blockhash: 0000f5f8e9262f5fcfb1044cba63a416728f05b181756146d894c1cdc650fcd9
blockheight: 71 winner: 417bb4ed50894b21207add3526caa834 blockhash: 00001c8d17580a1a15412b267448b816fee6a2c18eaa44eacdab9563ea9dfad8
blockheight: 72 winner: 323c63c511ff3330fe60026eb999a03c blockhash: 00015d21e88cb4a4fda768852776830c158200b75d63687ba467e66247eb232f
blockheight: 73 winner: b9a4eae1fddeddf713c775d7ff43ff99 blockhash: 000063bd807bb887d1425281ca59122e1d8e5681b84f7118704c7a575459d66b
blockheight: 74 winner: 417bb4ed50894b21207add3526caa834 blockhash: 000061d856c0ac2a103e4a0018287a07048531bf45183493f06391052957c11c
blockheight: 75 winner: 40f8329121414e071840d5ce0e826262 blockhash: 000187fd23623d2d10ea82524104c39599a400bab660386d63be08fb749ac5af
blockheight: 76 winner: faa5f30090ec9fde5226d0455becdc4a blockhash: 0